# Read from shapefile to PostGIS table

In [ ]:
# Import necessary libraries
import os
import glob
import psycopg2
import fiona
from shapely.geometry import Polygon

Let's read some of the necessary metadata from the shapefile. Metadata read includes the CRS, the attributes of the features. For this, we will use *fiona* to read the first feature (using context manager) from the shapefile and extract the metadata from it. 

In [ ]:
infname = glob.glob('vector/*.shp')[0]
with fiona.drivers():
    with fiona.open(infname) as src:
        geom = src.crs
        feat = src.next()
        attributes = feat['properties'].keys()
        gemtype = feat['geometry']['type']
        

Next, we connect to a postgres database (in this case, in the localhost)

In [ ]:
connection = psycopg2.connect(database="mydatabase",
                              user="myusername",
                              password="mypassword",
                              host="localhost")
cursor = connection.cursor()

Here, we will create a table in postgres. If we want to get a little bit fancy, we could automate the number and the names of the columns to correspond to the shapefile metatadata we read earlier into the variable *attributes*

In [ ]:
tablename = "Mclean_Dupage"
cursor.execute("DROP TABLE IF EXISTS {}".format(tablename))
cursor.execute("""
               CREATE TABLE {}
               (id SERIAL PRIMARY KEY,
               COUNTY_NAM VARCHAR NOT NULL,
               CO_FIPS BIGINT NOT NULL,
               geom GEOMETRY)""".format(tablename))
connection.commit()

Next, we will read each feature from the shapefile and write it as a postgres record: 

In [ ]:
with fiona.drivers():
    with fiona.open(infname) as src:
        for i in src:
            record = list(i['properties'].values())
            print ("{} county has {} fips".format(*record))
            geometry = Polygon(i['geometry']['coordinates'][0])
            wktgeometry = geometry.wkt
            cursor.execute("""
                   INSERT INTO {} (COUNTY_NAM, CO_FIPS, geom)
                   VALUES ('{}', {}, ST_GeomFromText('{}'))
                   """.format(tablename, *record, wktgeometry))
            connection.commit()

Next we assign coordinate reference system (CRS) to the PostGIS table:

In [ ]:
connection.commit()
cursor.execute("""
               ALTER TABLE {}
                   ALTER COLUMN geom TYPE geometry(Polygon, 4269)
                       USING ST_SetSRID(geom, 4269);
                       """.format(tablename))
connection.commit()

In [ ]:
# Close the connection
connection.close